# PubChem validation
This notebook demonstrates a new (?) idea in LBVS validation - pinging the pubchem server for assays for a given compound / target pair, and assessing the Outcome field. 

The main difficulty/novelty here is automating the approach while ensuring it gets the CORRECT assay. This is because many targets have different names, and there is no standard format for assay descriptions *or* outcomes.

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
from pprint import pprint

import re

from tqdm import tqdm_notebook

import scipy
from scipy import sparse
from scipy import stats

import pubchempy as pcp
from chembl_webresource_client.new_client import new_client 
import json
import requests

import copy

import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

import sys
sys.path.append("..")
import utils
import numpy as np

/Users/lmar3213/miniconda3/envs/lew_conda/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


# Make some structure-predictions with label correlations:



In [74]:
#all labels:
interaction_matrix = sparse.load_npz('../data/interaction_matrix_pchembl.npz')
smiles = pd.read_csv('../data/pchembl_chemicals.csv')
targets_df= pd.read_csv('../data/subset_targets.csv')

In [3]:
probability_matrix = utils.train_label_correlation(interaction_matrix)

  0%|          | 189/85681 [00:00<00:45, 1884.93it/s]

y_in shape is: (337951, 243)


100%|██████████| 337951/337951 [00:59<00:00, 5685.07it/s]


In [4]:
train, test, fps = utils.load_time_split(year=2030, 
                                         return_fingerprints=True) #set a year after now to get ALL records

probability_arr = probability_matrix.toarray()

In [5]:
##sort the predictions in order of probability, highest first. 

arr = probability_matrix.toarray()
arr = arr - interaction_matrix
arr_sorted = np.dstack(np.unravel_index(np.argsort(-arr.ravel()), (arr.shape[0], arr.shape[1])))[0]
print('Should be a high number < 1:')
print(probability_arr[arr_sorted[0][0]][arr_sorted[0][1]])
print('Should be a low number >= 0:')
print(probability_arr[arr_sorted[-1][0]][arr_sorted[-1][1]])
print('Sorted array indices:')
arr_sorted

Should be a high number < 1:
0.99991256
Should be a low number >= 0:
0.0
Sorted array indices:


array([[328242,    138],
       [ 68104,    138],
       [315225,     72],
       ...,
       [132327,     26],
       [132327,     28],
       [337950,    242]])

In [6]:
#sanity check - make sure the ligand IDX, smiles, and target IDX and target names line up:

for pair in arr_sorted[:10]:
    smi = smiles['canonical_smiles'].iloc[pair[0]]
    chembl_id = smiles['instance_id'].iloc[pair[0]]
    predicted_target = targets['pref_name'].iloc[pair[1]]
    tid = targets[targets['pref_name']==predicted_target]['pref_name'].iloc[0]
    print(smi[:10]+'...', predicted_target, '\t', tid)
    print(chembl_id, smi)

C[C@@H](Oc... Vascular endothelial growth factor receptor 2 	 Vascular endothelial growth factor receptor 2
CHEMBL601719 C[C@@H](Oc1cc(cnc1N)c2cnn(c2)C3CCNCC3)c4c(Cl)ccc(F)c4Cl
Nc1ncnc2sc... Vascular endothelial growth factor receptor 2 	 Vascular endothelial growth factor receptor 2
CHEMBL1998585 Nc1ncnc2scc(c3ccc(NC(=O)Nc4cc(ccc4F)C(F)(F)F)cc3)c12
CCN(CC)CCN... Serine/threonine-protein kinase PIM1 	 Serine/threonine-protein kinase PIM1
CHEMBL535 CCN(CC)CCNC(=O)c1c(C)[nH]c(\C=C\2/C(=O)Nc3ccc(F)cc23)c1C
CNC(=O)c1c... Vascular endothelial growth factor receptor 2 	 Vascular endothelial growth factor receptor 2
CHEMBL2005631 CNC(=O)c1cnc(N)c2c(csc12)c3ccc(NC(=O)Nc4cc(C)ccc4F)cc3
Cn1cc(cn1)... Vascular endothelial growth factor receptor 2 	 Vascular endothelial growth factor receptor 2
CHEMBL1988717 Cn1cc(cn1)c2cnn3c(N)c(cnc23)c4ccc(NC(=O)Nc5cccc(c5)C(F)(F)F)cc4
Cc1ccc(NC(... Vascular endothelial growth factor receptor 2 	 Vascular endothelial growth factor receptor 2
CHEMBL2000335 Cc1ccc

# Structure comparisons.

An important aspect of this is the knowledge of whether a prediction would have been predicted by nearest-neighbors anyway. These predictions aren't as interesting to us, which is the reason we use a structure-blind approach in the first place. 


The below functions help calculate jaccard distances in Morgan-space, and also calculate what rank a ligand-target pair would have been if just using the nearest neighbor approach. 

In [7]:
##The following is to calculate AVE bias:
def fast_jaccard(X, Y=None):
    """credit: https://stackoverflow.com/questions/32805916/compute-jaccard-distances-on-sparse-matrix"""
    if isinstance(X, np.ndarray):
        X = sparse.csr_matrix(X)
    if Y is None:
        Y = X
    else:
        if isinstance(Y, np.ndarray):
            Y = sparse.csr_matrix(Y)
    assert X.shape[1] == Y.shape[1]

    X = X.astype(bool).astype(int)
    Y = Y.astype(bool).astype(int)
    intersect = X.dot(Y.T)
    x_sum = X.sum(axis=1).A1
    y_sum = Y.sum(axis=1).A1
    xx, yy = np.meshgrid(x_sum, y_sum)
    union = ((xx + yy).T - intersect)
    return (1 - intersect / union).A


# PubChem-pinging 

This became much cleaner when wrapped in a class. The below is a class to perform a number of functions:

- save/load checkpoints - i.e. if something breaks, or you want to come back to this later after shutting the laptop, you can save all the data and load it afterwards to pick back up
- ping `pubchempy` for compound ID (CID) numbers.
- ping `chembl_webresource_client` for target synonyms
- ping pubchem REST API for assays associated with a CID
- records all of this stuff in dicts to be saved as JSON so I can parse it later. 

In [168]:
import json

class PubChemValidator(object):
    def __init__(self, targets_df, interaction_matrix, fps):
        self.tdf = targets_df
        self.interaction_matrix = interaction_matrix
        self.fps = fps

        self.ligands = {}
        self.targets = {}
        self.predictions = {}
    
    def load_checkpoint(self):
        self.ligands = json.load(open('ligands.json', 'r'))
        self.targets = json.load(open('targets.json', 'r'))
        self.predictions = json.load(open('predictions.json', 'r'))
        
    def save_checkpoint(self):
        with open('ligands.json', 'w') as fp:
            json.dump(self.ligands, fp)
        with open('targets.json', 'w') as fp:
            json.dump(self.ligands, fp)
        with open('predictions.json', 'w') as fp:
            json.dump(self.predictions, fp)
            
    def has_ligand(self, idx):
        return str(idx) in self.ligands
    
    def has_target(self, idx):
        return str(idx) in self.targets

    def has_prediction(self, l_idx, t_idx):
        return str(l_idx)+':'+str(t_idx) in self.predictions
    
    def create_prediction(self, l_idx, t_idx, prob):
        record = dict()
        record['prob'] = str(prob)
        nn = self.get_nnrank_of_target(l_idx, t_idx)
        record['nn'] = nn
        
        self.predictions[str(l_idx)+':'+str(t_idx)] = record
        
    def create_target(self, idx):
        self.targets[str(idx)] = dict()
        record = self.targets[str(idx)]
    
        pref_name = self.tdf['pref_name'].iloc[idx]
        tid = self.tdf[self.tdf['pref_name']==pref_name]['chembl_id'].iloc[0]
        synonyms = get_synonyms(tid)
    
        record['pref_name'] = pref_name
        record['tid'] = tid
        record['synonyms'] = synonyms
        
    def create_ligand(self, idx):
        self.ligands[str(idx)] = dict()
        record = self.ligands[str(idx)]
    
        smi = smiles['canonical_smiles'].iloc[idx]
        chemblid = smiles['instance_id'].iloc[idx]
        cid = self.get_cid(smi)
        assays = self.get_assay_summary(cid)
        assays_parsed = self.parse_assays(assays)
    
        record['smi']=smi
        record['chemblid'] = chemblid
        record['cid'] = cid
        record['assays'] = assays_parsed
        
    def get_cid(self, smi):
        try:
            c = pcp.get_compounds(smi, 'smiles')[0]
            return c.cid
        except Exception as e:
            print(e)
            return 'cid_failed'
        
    def get_synonyms(self, tid):
        target = new_client.target
        res = target.filter(target_chembl_id=tid)
        synonyms = [i['component_synonym'] for i in res[0]['target_components'][0]['target_component_synonyms']]
        #clean:
        synonyms = [self.clean_text(i) for i in target_synonyms]
        return synonyms
    
    def clean_text(self, input_string):
        #source: https://stackoverflow.com/questions/34860982/replace-the-punctuation-with-whitespace
        #replace these with whitespace:
        clean_string = re.sub(r"""
               [(),.;@#?!&$]+  # Accept one or more copies of punctuation
               \ *           # plus zero or more copies of a space,
               """,
               " ",          # and replace it with a single space
               input_string.lower(), flags=re.VERBOSE)
    
        #replace these with nothing:
        clean_string = clean_string.replace('-', ' ')
        clean_string = clean_string.replace('=', '')
        return clean_string

    def get_assay_summary(self, cid):
        b = json.loads(requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/'+str(cid)+'/assaysummary/JSON').content)
        return b
    
    def parse_assays(self, assays):
        assays_parsed = []
        for assay in assays['Table']['Row']:
            cell = assay['Cell']
            aid = cell[0]
            name = self.clean_text(cell[11])
            activity = cell[6]
            
            assays_parsed.append([aid, activity, name])
        return assays_parsed
    
    def get_nnrank_of_target(self, ligand_idx, target_idx):
        
        positives = self.interaction_matrix[ligand_idx].nonzero()[1]
        all_distances = fast_jaccard(self.fps[ligand_idx], self.fps)[0]
        s = np.argsort(all_distances)
    
        pred = target_idx
        curr_rank = 0
        count=1
        preds = []
        seen = []

        while pred not in preds:
            predictions = self.interaction_matrix[s[count]].nonzero()[1]
    
            preds = np.setdiff1d(predictions,positives)
            preds = np.setdiff1d(preds, seen)
            seen += list(preds)
            curr_rank += 0 if len(preds)<1 else np.mean(np.arange(len(preds))+1)

            count+=1
        return curr_rank
            
            
        

    

In [217]:
pcv = PubChemValidator(targets_df, interaction_matrix, fps)

In [218]:
#take a random sample of assays across the (many) higher probability ones:
n = 400_000
take = 20000

weight = 1 / (np.arange(n)/take+1)
weight = weight/weight.sum()

sample = np.random.choice(n, take, p=weight, replace=False)

In [ ]:
for count, prediction in enumerate(arr_sorted[sample]):
    print(prediction)
    ligand_idx = prediction[0]
    target_idx = prediction[1]
    probability = probability_arr[ligand_idx][target_idx]
    
    try:
        if not pcv.has_ligand(ligand_idx):
            pcv.create_ligand(ligand_idx)
        
        if not pcv.has_target(target_idx):
            pcv.create_target(target_idx)
        
        if not pcv.has_prediction(ligand_idx, target_idx):
            pcv.create_prediction(ligand_idx, target_idx, probability)
        
    except KeyboardInterrupt:
        raise
    except Exception:
        pass
    
    if count>0 and count%50==0:
        pcv.save_checkpoint()

[309137     48]
[5148   30]
[2691   99]
[224471     86]
[63560   169]
[147393    177]
[35497   225]
[333416     15]
[319186    102]
[126407     71]
[268263    178]
[160711     52]
[311062    205]
[5421   15]
[88082    13]
[309907     86]
[16196   138]
[216348     52]
[116726     82]
[66753   136]
[205032    100]
[129385     17]
[334454     83]
[120980    220]
[293264     83]
[335039    138]
[90967   194]
[73885   122]
[26355   225]
[129028    122]
[309217    196]
[276400    158]
[67495    16]
[19371    38]
[133980    177]
[67472   189]
[308250     84]
[222231    178]
